# Prototype_2
### 추출해와서 출력하기

#### Requirements

In [1]:
import os
import glob

import numpy as np
import pandas as pd
import json

import mediapipe as mp
import cv2
import pytube

objc[1094]: Class CaptureDelegate is implemented in both /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x11cdfc860) and /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x281ea2538). One of the two will be used. Which one is undefined.
objc[1094]: Class CVWindow is implemented in both /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x10fea4a68) and /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x281ea2588). One of the two will be used. Which one is undefined.
objc[1094]: Class CVView is implemented in both /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x10fea4a90) and /Users/nuyhc/miniforge3/envs/Deep/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x281ea25b0). One of the two will be used. Which one is undefined.
objc[1094]: Class CV

#### Prototype

In [84]:
class JustDDance:
    def __init__(self):
        self.__download_path = "./video/"
        self.__keyporint_extraction_path = "./keypoint_extraction/"
        self.__dance_fps = None
        self.__dance_shape = None
        self.__user_fps = None
        self.__user_shape = None
    
    def download_video(self):
        url = input("연습할 춤의 유튜브 링크: ")
        yt = pytube.YouTube(url)
        stream = yt.streams.filter(res="720p").first()
        stream.download(self.__download_path)
    
    def scaling_coor(self, keypoint_path): # "./keypoint_extraction/[주간아 직캠] IVE YUJIN - LOVE DIVE (아이브 유진 - 러브 다이브) l EP556_keypoints.json"
        with open(keypoint_path, "r") as file:
            data = json.load(file)
        pose_cor = pd.DataFrame(data["pose"])
        return np.array(pose_cor)
    
    def extract_keypoint(self, video_path):
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_style = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic
        
        keypoint_dict_pose = []
        keypoint_dict_left_hand = []
        keypoint_dict_right_hand = []
        keypoint_dict = {}
        
        try: cap = cv2.VideoCapture(video_path)
        except: return -1
        
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                success, image = cap.read()
                if not success: break
                self.__dance_shape = image.shape
                self.__dance_fps = cap.get(cv2.CAP_PROP_FPS)
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                try:
                    keypoint_dict_pose.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.pose_landmarks.landmark)})
                    keypoint_dict_left_hand.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.left_hand_landmarks.landmark)})
                    keypoint_dict_right_hand.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.right_hand_landmarks.landmark)})
                except:
                    keypoint_dict_pose.append({str(idx): [0, 0, 0] for idx in range(0, 33)})
                    keypoint_dict_left_hand.append({str(idx): [0, 0, 0] for idx in range(0, 21)})
                    keypoint_dict_right_hand.append({str(idx): [0, 0, 0] for idx in range(0, 21)})
                    
                keypoint_dict = {"pose": keypoint_dict_pose, "left_hand": keypoint_dict_left_hand, "right_hand": keypoint_dict_right_hand}
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        with open(self.__keyporint_extraction_path+video_path.split("/")[2].split(".")[0]+"_keypoints.json", "w") as fp:
            json.dump(keypoint_dict, fp)
            
    def show_dance_tutorial(self):
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_style = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic
        
        try: cap = cv2.VideoCapture(0)
        except: cap = cv2.VideoCapture(1)
        load_dance = cv2.VideoCapture("./video/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561.mp4")
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        dance_cors = self.scaling_coor("./keypoint_extraction/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561_keypoints.json")
        dance_cors_fps = 0
        extract_points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
        cv2.startWindowThread()
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                success, image = cap.read()
                ret, dance = load_dance.read()
                if not success: break
                self.__user_shape = image.shape
                self.__user_fps = cap.get(cv2.CAP_PROP_FPS)
                self.__dance_shape = dance.shape
                self.__dance_fps = load_dance.get(cv2.CAP_PROP_FPS)

                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                try:
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=5, circle_radius=1),
                                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=3, circle_radius=1))
                    skeleton = {}
                    # for pose_point in range(33):
                    for pose_point in extract_points:
                        scale_x_cor_pose, scale_y_cor_pose = int(dance_cors[dance_cors_fps][pose_point][0]*self.__user_shape[1]), int(dance_cors[dance_cors_fps][pose_point][1]*self.__user_shape[0])
                        cv2.circle(image, (scale_x_cor_pose, scale_y_cor_pose), 10, (204, 0, 102), cv2.FILLED)
                        skeleton[pose_point] = (scale_x_cor_pose, scale_y_cor_pose)

                        tn_x, tn_y, tn_z = dance_cors[dance_cors_fps][pose_point][0:3]
                        print(results.pose_landmarks.landmark)
                    dance_cors_fps += 1
                    cv2.line(image, skeleton[16], skeleton[14], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오손목 -> 오팔꿈치
                    cv2.line(image, skeleton[14], skeleton[12], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오팔꿈치 -> 오어깨
                    cv2.line(image, skeleton[12], skeleton[11], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오어깨 -> 왼어깨
                    cv2.line(image, skeleton[11], skeleton[13], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼어깨 -> 왼팔꿈치
                    cv2.line(image, skeleton[13], skeleton[15], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼팔꿈치 -> 왼손목
                    cv2.line(image, skeleton[12], skeleton[24], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오어깨 -> 오엉덩이
                    cv2.line(image, skeleton[24], skeleton[26], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오엉덩이 -> 오무릎
                    cv2.line(image, skeleton[26], skeleton[28], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 오무릎 -> 오발목
                    cv2.line(image, skeleton[11], skeleton[23], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼어깨 -> 왼엉덩이
                    cv2.line(image, skeleton[23], skeleton[25], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼엉덩이 -> 왼무릎
                    cv2.line(image, skeleton[25], skeleton[27], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼무릎 -> 왼발목
                    cv2.line(image, skeleton[24], skeleton[23], (204, 0, 102), thickness=5, lineType=cv2.LINE_AA, shift=None) # 왼엉덩이 -> 오엉덩이

                    # Acc

                except:
                    break
                # TODO: 싱크 문제 해결
                # try:
                #     h_output = np.hstack((cv2.flip(dance, 1), image))
                # except:
                #     h_output = np.hstack((np.zeros((512, 512, 3), np.uint8), image))
                cv2.imshow("Just DDance!", image)
                if cv2.waitKey(1)&0xFF==ord("q"): break
        load_dance.release()        
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
                


In [85]:
jd = JustDDance()

In [ ]:
jd.extract_keypoint("./video/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561.mp4")

In [86]:
jd.show_dance_tutorial()

In [73]:
temp = []
for f, s in zip([1, 2, 3], [1, 1, 1]):
    temp.append(s-f)

In [74]:
np.linalg.norm(temp)

2.23606797749979